In [1]:
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import cv2
import numpy as np
import densenet as dn
import pdb
# used for logging to TensorBoard
from tensorboard_logger import configure, log_value

In [14]:
#model = dn.DenseNet3(100, 4, 24, reduction=args.reduce,bottleneck=args.bottleneck, dropRate=args.droprate, small_inputs = False)
parser = argparse.ArgumentParser(description='PyTorch DenseNet Training')
parser.add_argument('--epochs', default=510, type=int,
                    help='number of total epochs to run')
parser.add_argument('--start-epoch', default=500, type=int,
                    help='manual epoch number (useful on restarts)')
parser.add_argument('-b', '--batch-size', default=5, type=int,
                    help='mini-batch size (default: 32)')
parser.add_argument('--lr', '--learning-rate', default=0.01, type=float,
                    help='initial learning rate')
parser.add_argument('--momentum', default=0.9, type=float, help='momentum')
parser.add_argument('--weight-decay', '--wd', default=1e-4, type=float,
                    help='weight decay (default: 1e-4)')
parser.add_argument('--print-freq', '-p', default=10, type=int,
                    help='print frequency (default: 10)')
parser.add_argument('--layers', default=100, type=int,
                    help='total number of layers (default: 100)')
parser.add_argument('--growth', default=12, type=int,
                    help='number of new channels per layer (default: 12)')
parser.add_argument('--droprate', default=0, type=float,
                    help='dropout probability (default: 0.0)')
parser.add_argument('--no-augment', dest='augment', action='store_false',
                    help='whether to use standard augmentation (default: False)')
parser.add_argument('--reduce', default=0.5, type=float,
                    help='compression rate in transition stage (default: 0.5)')
parser.add_argument('--no-bottleneck', dest='bottleneck', action='store_false',
                    help='To not use bottleneck block')
parser.add_argument('--resume', default='C:\\Users\\fs\\Desktop\\densenet-pytorch-master\\runs_k=12\\DenseNet_Unet_fs\\model_best.pth', type=str,
                    help='path to latest checkpoint (default: none)')
parser.add_argument('--name', default='DenseNet_Unet_fs', type=str,
                    help='name of experiment')
parser.add_argument('--tensorboard',
                    help='Log progress to TensorBoard', action='store_true')
parser.set_defaults(bottleneck=True)
parser.set_defaults(augment=False)
global args, best_prec1
args = parser.parse_args([])
val_dirs = 'C:\\Users\\fs\\Desktop\\densenet-pytorch-master\\testimg'

In [15]:
normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                     std=[x/255.0 for x in [63.0, 62.1, 66.7]])

transform_train = transforms.Compose([
        transforms.ToTensor(),    
        normalize,
            ])


In [16]:
model = dn.DenseNet3(100, 4, 12, bottleneck=args.bottleneck,  small_inputs = False)
model = model.cuda()
if args.resume:
    if os.path.isfile(args.resume):
        print("=> loading checkpoint '{}'".format(args.resume))
        checkpoint = torch.load(args.resume)
        args.start_epoch = checkpoint['epoch']
        #best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        print("=> loaded checkpoint '{}' (epoch {})"
                .format(args.resume, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(args.resume))

=> loading checkpoint 'C:\Users\fs\Desktop\densenet-pytorch-master\runs_k=12\DenseNet_Unet_fs\model_best.pth'
=> loaded checkpoint 'C:\Users\fs\Desktop\densenet-pytorch-master\runs_k=12\DenseNet_Unet_fs\model_best.pth' (epoch 296)


In [17]:
from PIL import Image

In [18]:
%%time
img = Image.open('.\\testimg\\num4.png')
if img.mode != 'RGB':
            img = img.convert('RGB')
transform_i = transforms.Compose([    
        transforms.ToTensor(),
        normalize,
            ])
input = transform_i(img)
input = input.unsqueeze(0)
input = input.cuda()
input = torch.autograd.Variable(input)
model.eval()
output = model(input)
print(output)
x= torch.max(output,1)
classes = []
with open('classes.txt', 'r') as list_:
    for line in list_:
        classes.append(line.rstrip('\n'))

print('Prediction: ', str(classes[x[1]]))

tensor([[-4.1626, 13.5673, -7.3540, -2.0392]], device='cuda:0',
       grad_fn=<AddmmBackward>)
Prediction:  flaw
Wall time: 67 ms


In [40]:
img = Image.open('.\\testimg\\flaw.jpg')
ii = img.crop((100,150,200,200))

In [41]:
ii.size

(100, 50)

In [22]:
input = input.unsqueeze(0)
input = input.cuda()
input = torch.autograd.Variable(input)

output = model(input)
print(output)
x= torch.max(output,1)
classes = []
with open('classes.txt', 'r') as list_:
    for line in list_:
        classes.append(line.rstrip('\n'))

print('Prediction: ', str(classes[x[1]]))

tensor([[ 7.2650, -1.9809, -2.6503, -2.6371]], device='cuda:0',
       grad_fn=<AddmmBackward>)
Prediction:  bubble


In [27]:
%%time
img = cv2.imread('bubble(3).bmp').astype(np.float32)
#img = cv2.imread('bubble(2).bmp').astype(np.float32)
img.shape
img = np.transpose(img, axes=[2, 0, 1])

#img = np.expand_dims(img, axis=0)
print(type(img))
transform_t = transforms.Compose([    
        normalize,
            ])
#type(input)
inputim = torch.from_numpy(img)
inputim1 = transform_t(inputim/255)
inputim1 = inputim1.unsqueeze(0)
inputim1 = inputim1.cuda()
inputim1 = torch.autograd.Variable(inputim1)
print(inputim1.shape)

#model.eval()
output = model(inputim1)
print(output)
x= torch.max(output,1)
classes = []
with open('classes.txt', 'r') as list_:
    for line in list_:
        classes.append(line.rstrip('\n'))

print('Prediction: ', str(classes[x[1]]))

<class 'numpy.ndarray'>
torch.Size([1, 3, 200, 200])
tensor([[ 7.2650, -1.9809, -2.6503, -2.6371]], device='cuda:0',
       grad_fn=<AddmmBackward>)
Prediction:  bubble
Wall time: 68 ms


In [ ]:
val_dataset = datasets.ImageFolder(
        val_dirs,
        transform_train)   
kwargs = {'num_workers': 0, 'pin_memory': True}
val_loader = torch.utils.data.DataLoader(
        #datasets.CIFAR10('../data', train=False, transform=transform_test),
        val_dataset,
        batch_size=args.batch_size, shuffle=True, **kwargs)

In [27]:
model.eval()
for i, (input, target) in enumerate(val_loader):
        target = target.cuda(async=True)
        print(type(input))
        input = input.cuda()
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)

        # compute output
        output = model(input_var)

<class 'torch.Tensor'>


C:\Users\fs\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\fs\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
